In [3]:
from geopandas import GeoDataFrame as df
import math
from shapely.geometry import LineString,Point
import json
%matplotlib inline
from geopandas import GeoSeries as ds
import pandas as pd

#from shape generate lines

In [4]:
shapes=df.from_csv('shapes.txt',index_col=None)
shapes['shape_seq']=shapes['shape_pt_sequence'].apply(lambda x: math.floor(x/10000))
shapes['shape_seq_id']=shapes[['shape_id','shape_seq']].apply(lambda x:str(x[0])+'-'+str(int(x[1])),axis=1)

In [5]:
distances=ds.from_csv('mta_shapes_meters_2014.csv',header=None)
speeds=ds.from_csv('speed.txt',sep='\t',header=None)
trips=df.from_csv('trips.txt')

In [6]:
line_dict={}
for shape_id in set(shapes.shape_seq_id):
    try:
        line=LineString(zip(shapes[shapes.shape_seq_id==shape_id]['shape_pt_lon'].values.tolist(),shapes[shapes.shape_seq_id==shape_id]['shape_pt_lat'].values.tolist()))
        speed=speeds[shape_id]
        distance=distances[shape_id]
        shape_num=shape_id.split('-')[0]
        route=list(set(trips[trips.shape_id==shape_num].index))[0]
        line_dict[shape_id]={}
        line_dict[shape_id]['speed']=speed
        line_dict[shape_id]['distance']=distance
        line_dict[shape_id]['geometry']=line
        line_dict[shape_id]['route_id']=route
    except:
        pass

In [25]:
json_dump=df(line_dict).transpose()
json_dump.reset_index(inplace=True)
json_dump['shape_id']=json_dump['index'].apply(lambda x:x.split('-')[0])
json_dump['shape_seq']=json_dump['index'].apply(lambda x:x.split('-')[1])
json_dump.drop('index', axis=1, inplace=True)

In [26]:
with open('brooklyn_0804_speed_distance.geojson','w') as f:
    f.write(json_dump.to_json())

#dump to redis

In [27]:
import redis
pool = redis.ConnectionPool(host='busvis.cloudapp.net', port=6379, db=10)
r = redis.Redis(connection_pool=pool)

In [ ]:
for i in set(json_dump.route_id):
    r.hset('speed_distance',i,json_dump[json_dump.route_id==i].to_json())

#test

In [ ]:
eval(r.hget('speed_distance','B38'))

In [ ]:
df.from_file('brooklyn_0804_speed_distance.geojson')